In [1]:
%pip install black docarray langchain tiktoken

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os

from pathlib import Path

openai_api_key_path = Path("D:\\") / "proj" / "llm" / ".openai_key"
with open(openai_api_key_path, "r", encoding="utf-8") as f:
    os.environ["OPENAI_API_KEY"] = f.read().strip()

In [3]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template("tell me a short joke about {topic}")
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

print(chain.invoke({"topic": "huskies"}))

Sure, here you go: 

Why did the Husky bring a ladder to the dog park? 

Because it wanted to reach new heights and become the top dog!


In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["harrison worked at kensho", "bears like to eat honey", "huskies are cute", "harrison likes cute things"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-3.5-turbo")
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

print(chain.invoke("Does Harrison like huskies? Yes or no?"))

Yes.


## Gumshoe Quest

In [71]:
from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    [
        "The user is investigating a stolen manuscript",
        "You and the user are in Evelyn Hart's home office",
        "The office contains a large wooden desk, a window, and a control panel for the security system",
        "The desk is covered in papers and books. There is also a calendar on the desk.",
        "CLUE: If the user looks at the door, they will find that there were no signs of forced entry",
        'CLUE: If the user looks through the papers on the desk, they will find a cryptic note. It reads: "The best stories are worth stealing."',
        "CLUE: The window is not locked. In fact, it is slightly open, suggesting a possible entry point",
        "CLUE: If the user asks to check the logs for the security system, they will find that it was disabled and not active at the time of the theft",
        "CLUE: If the user asks to look at the calendar for the time when the manuscript was stolen, they will find that Evelyn was out to lunch with her editor at the time",

    ],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()


template = """The user will instruct tell you that he wants to do something relating to investigating the location.
You should respond based on the following context but omit information unless the user specifically asks.
Any piece of context labeled CLUE should only be revelaed if the user asks to investigate closely or specifically asks about it.
If the user asks something that causes you to reveal a clue, prefex the response with 'CLUE:'.

Context:
{context}

Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI(model="gpt-4")
output_parser = StrOutputParser()
setup_and_retrieval = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
)
chain = setup_and_retrieval | prompt | model | output_parser

In [72]:
print(chain.invoke("Where am I?"))

You are in Evelyn Hart's home office.


In [73]:
print(chain.invoke("What objects are in the room?"))

The room contains a large wooden desk, a window, and a control panel for the security system. The desk is covered in papers, books, and a calendar.


In [74]:
print(chain.invoke("What is on the desk?"))

The desk is covered in papers and books. There is also a calendar on the desk.


In [75]:
print(chain.invoke("Is the window locked?"))

CLUE: The window is not locked. In fact, it is slightly open, suggesting a possible entry point.


In [76]:
print(chain.invoke("Look closely through the papers on the desk. Is there anything out of the ordinary?"))

CLUE: Yes, you find a cryptic note among the papers. It reads: "The best stories are worth stealing."


In [69]:
# print(chain.invoke("What is up with the security system?"))
print(chain.invoke("Does the security system have logs?"))

Yes, the security system does have logs.


In [77]:
print(chain.invoke("Check the logs for the security system. Was the security system active at the time the manuscript was stolen?"))

CLUE: The logs for the security system reveal that it was disabled and not active at the time of the theft.


In [78]:
print(chain.invoke("Check the door. Was there any sign of forced entry?"))

CLUE: Upon investigating the door, there were no signs of forced entry.
